In [1]:
# imports
import io
import sys
import os
import subprocess
import requests
from IPython.display import Markdown, display, update_display
from huggingface_hub import login
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer, BitsAndBytesConfig, AutoModelForSpeechSeq2Seq, AutoProcessor, pipeline
import gradio as gr
import torch
import ollama
import platform

In [2]:
# will be trying with 2 models on ollama and the alibaba qwen model from fugging face
ollama_code_llama = "codellama:7b"
ollama_code_gemma = "codegemma:latest"

In [3]:
system_message = "You are a brilliant software developer assistant that reimplements Python code in high performance C++. "
system_message += "Respond only with C++ code; use comments sparingly and do not provide any explanation other than occasional comments. "
system_message += "The C++ response needs to produce an identical output in the fastest possible time. "
system_message += "Keep implementations of random number generators identical so that results match exactly."

In [4]:
def user_prompt_for(python):
    user_prompt = "Rewrite this Python code in C++ with the fastest possible implementation that produces identical output in the least time. "
    user_prompt += "Respond only with C++ code; do not explain your work so dont use comments. "
    user_prompt += "Pay attention to number types to ensure no int overflows. Remember to #include all necessary C++ packages such as iomanip.\n\n"
    user_prompt += python
    return user_prompt

In [5]:
def messages_for(python):
    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_prompt_for(python)}
    ]

In [6]:
# write to a file called final.cpp

def write_output(cpp):
    code = cpp.replace("```cpp","").replace("```","")
    with open("final.cpp", "w") as f:
        f.write(code)

In [7]:
def ollama_response(python, model):    
    streamingResponse = ollama.chat(model=model, messages=messages_for(python),stream=True)
    response = ""
    for chunk in streamingResponse:
        response += chunk.get('message', {}).get('content', '') or ''
       # response = response.replace("```","").replace("markdown", "")
        yield response

In [8]:
def hf_response(python, model):    
    streamingResponse = ollama.chat(model=model, messages=messages_for(python),stream=True)
    response = ""
    for chunk in streamingResponse:
        response += chunk.get('message', {}).get('content', '') or ''
      #  response = response.replace("```","").replace("markdown", "")
        yield response

In [9]:
python_sample_options = ["Hello, World", "Calculate pi", "hard python", "Sieve of Eratosthenes"]

python_code_samples = {
    python_sample_options[0]: """
import time

start_time = time.time()

print("Hello, world")

end_time = time.time()

print(f"Execution Time: {(end_time - start_time):.6f} seconds")
""",

    python_sample_options[1]:  """
import time

def calculate(iterations, param1, param2):
    result = 1.0
    for i in range(1, iterations+1):
        j = i * param1 - param2
        result -= (1/j)
        j = i * param1 + param2
        result += (1/j)
    return result

start_time = time.time()
result = calculate(100_000_000, 4, 1) * 4
end_time = time.time()

print(f"Result: {result:.12f}")
print(f"Execution Time: {(end_time - start_time):.6f} seconds")
""",

    python_sample_options[2]:  """
# Be careful to support large number sizes

def lcg(seed, a=1664525, c=1013904223, m=2**32):
    value = seed
    while True:
        value = (a * value + c) % m
        yield value
        
def max_subarray_sum(n, seed, min_val, max_val):
    lcg_gen = lcg(seed)
    random_numbers = [next(lcg_gen) % (max_val - min_val + 1) + min_val for _ in range(n)]
    max_sum = float('-inf')
    for i in range(n):
        current_sum = 0
        for j in range(i, n):
            current_sum += random_numbers[j]
            if current_sum > max_sum:
                max_sum = current_sum
    return max_sum

def total_max_subarray_sum(n, initial_seed, min_val, max_val):
    total_sum = 0
    lcg_gen = lcg(initial_seed)
    for _ in range(20):
        seed = next(lcg_gen)
        total_sum += max_subarray_sum(n, seed, min_val, max_val)
    return total_sum

# Parameters
n = 10000         # Number of random numbers
initial_seed = 42 # Initial seed for the LCG
min_val = -10     # Minimum value of random numbers
max_val = 10      # Maximum value of random numbers

# Timing the function
import time
start_time = time.time()
result = total_max_subarray_sum(n, initial_seed, min_val, max_val)
end_time = time.time()

print("Total Maximum Subarray Sum (20 runs):", result)
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
""",

    python_sample_options[3]:  """
import time
start_time = time.time()
stop_at=100_000_000
prime = [True] * (stop_at + 1)
p = 2

while p * p <= stop_at:
    # If prime[p] is True, then p is a prime
    if prime[p]:
        # Mark all multiples of p as non-prime
        for i in range(p * p, stop_at + 1, p):
            prime[i] = False
    p += 1

# Collect all prime numbers
primes = [p for p in range(2, stop_at + 1) if prime[p]]

end_time = time.time()
print("Maximum prime:, {:,}".format(primes[-1]))
print("Execution Time: {:.6f} seconds".format(end_time - start_time))
"""
}

In [10]:
def run_model(python, model):
    if model=="CODELLAMA":
        result = ollama_response(python,ollama_code_llama)
    elif model=="CODEGEMMA":
        result = ollama_response(python,ollama_code_gemma)
    else:
        raise ValueError("Unknown model")
    for stream_so_far in result:
        yield stream_so_far   

In [11]:
def execute_python(code):
    try:
        output = io.StringIO()
        sys.stdout = output
        exec(code)
    finally:
        sys.stdout = sys.__stdout__
    return output.getvalue()

In [12]:
def execute_cpp(code):
    write_output(code)
    try:
        compile_result = subprocess.run(compiler_cmd[2], check=True, text=True, capture_output=True)
        run_cmd = ["./final"]
        run_result = subprocess.run(run_cmd, check=True, text=True, capture_output=True)
        return run_result.stdout
    except subprocess.CalledProcessError as e:
        return f"An error occurred:\n{e.stderr}"

In [13]:
import platform

VISUAL_STUDIO_2022_TOOLS = "C:\\Program Files\\Microsoft Visual Studio\\2022\\Community\\Common7\Tools\\VsDevCmd.bat"
VISUAL_STUDIO_2019_TOOLS = "C:\\Program Files (x86)\\Microsoft Visual Studio\\2019\\BuildTools\\Common7\\Tools\\VsDevCmd.bat"

simple_cpp = """
#include <iostream>

int main() {
    std::cout << "Hello";
    return 0;
}
"""

def run_cmd(command_to_run):
    try:
        run_result = subprocess.run(command_to_run, check=True, text=True, capture_output=True)
        return run_result.stdout if run_result.stdout else "SUCCESS"
    except Exception as e:
        #print("An error occurred:", e)
        return e

def c_compiler_cmd(filename_base):
    my_platform = platform.system()
    my_compiler = []
    
    try:
        with open("simple.cpp", "w") as f:
            f.write(simple_cpp)
            
        if my_platform == "Windows":
            if os.path.isfile(VISUAL_STUDIO_2022_TOOLS):
                if os.path.isfile("./simple.exe"):
                   os.remove("./simple.exe")
                compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", "simple.cpp"]
                result = run_cmd(compile_cmd)
                #print("------")
                #print(result)
                #print("------")
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple.exe"]) == "Hello":
                         my_compiler = ["Windows", "Visual Studio 2022", ["cmd", "/c", VISUAL_STUDIO_2022_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
        
            if not my_compiler:
                if os.path.isfile(VISUAL_STUDIO_2019_TOOLS):
                    if os.path.isfile("./simple.exe"):
                        os.remove("./simple.exe")
                    compile_cmd = ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", "simple.cpp"]
                    if run_cmd(compile_cmd):
                        if run_cmd(["./simple.exe"]) == "Hello":
                            my_compiler = ["Windows", "Visual Studio 2019", ["cmd", "/c", VISUAL_STUDIO_2019_TOOLS, "&", "cl", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
                
        elif my_platform == "Linux":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["g++", "simple.cpp", "-o", "simple"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Linux", "GCC (g++)", ["g++", f"{filename_base}.cpp", "-o", f"{filename_base}" ]]
    
            if not my_compiler:
                if os.path.isfile("./simple"):
                    os.remove("./simple")
                compile_cmd = ["clang++", "simple.cpp", "-o", "simple"]
                if run_cmd(compile_cmd):
                    if run_cmd(["./simple"]) == "Hello":
                        my_compiler = ["Linux", "Clang++", ["clang++", f"{filename_base}.cpp", "-o", f"{filename_base}"]]
        
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    
        elif my_platform == "Darwin":
            if os.path.isfile("./simple"):
                os.remove("./simple")
            compile_cmd = ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", "simple", "simple.cpp"]
            if run_cmd(compile_cmd):
                if run_cmd(["./simple"]) == "Hello":
                    my_compiler = ["Macintosh", "Clang++", ["clang++", "-Ofast", "-std=c++17", "-march=armv8.5-a", "-mtune=apple-m1", "-mcpu=apple-m1", "-o", f"{filename_base}", f"{filename_base}.cpp"]]
    
            if not my_compiler:
                my_compiler=[my_platform, "Unavailable", []]
    except:
        my_compiler=[my_platform, "Unavailable", []]
        
    if my_compiler:
        return my_compiler
    else:
        return ["Unknown", "Unavailable", []]


In [14]:
css = """
.python {background-color: #306998;}
.cpp {background-color: #050;}
"""

In [19]:
compiler_cmd = c_compiler_cmd("final")
with gr.Blocks(css=css) as ui:
    gr.Markdown("## Convert code from Python to C++")
    with gr.Row():
        python = gr.Textbox(label="Python code:", value=python_code_samples[python_sample_options[1]], lines=10)
        cpp = gr.Textbox(label="C++ code:", lines=10)
    with gr.Row():
        model = gr.Dropdown(["CODELLAMA", "CODEGEMMA"], label="Select model", value="CODELLAMA")
    with gr.Row():
        convert = gr.Button("Convert code")
    with gr.Row():
        python_run = gr.Button("Run Python")
        cpp_run = gr.Button("Run C++")
    with gr.Row():
        python_out = gr.TextArea(label="Python result:", elem_classes=["python"])
        cpp_out = gr.TextArea(label="C++ result:", elem_classes=["cpp"])

    convert.click(run_model, inputs=[python, model], outputs=[cpp])
    python_run.click(execute_python, inputs=[python], outputs=[python_out])
    cpp_run.click(execute_cpp, inputs=[cpp], outputs=[cpp_out])

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863

To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "C:\Users\Darks\anaconda3\envs\llms\Lib\site-packages\gradio\queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Darks\anaconda3\envs\llms\Lib\site-packages\gradio\route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Darks\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 2042, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Darks\anaconda3\envs\llms\Lib\site-packages\gradio\blocks.py", line 1589, in call_function
    prediction = await anyio.to_thread.run_sync(  # type: ignore
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\Darks\anaconda3\envs\llms\Lib\site-packages\anyio\to_thread.py", line 56, in run_sync
    return awai

In [18]:
# import subprocess
# import os

# def run_cmd(command_to_run):
#     try:
#         result = subprocess.run(command_to_run, check=True, shell=True,
#                               capture_output=True, text=True, encoding='utf-8')
#         return result.stdout.strip()
#     except subprocess.CalledProcessError as e:
#         return f"Error: {e.stderr.strip()}"

# def c_compiler_cmd(filename_base):
#     vs2022_bat = r"C:\Program Files\Microsoft Visual Studio\2022\Community\Common7\Tools\VsDevCmd.bat"
    
#     if os.path.exists(vs2022_bat):
#         # Compilation command
#         compile_cmd = f'""{vs2022_bat}" && cl /nologo simple.cpp /Fesimple.exe"'
        
#         # Execute compilation
#         compile_result = run_cmd(f'cmd /c {compile_cmd}')
        
#         # Verify executable
#         if os.path.exists("simple.exe"):
#             output = run_cmd("simple.exe")
#             if "Hello" in output:
#                 return ["Windows", "Visual Studio 2022", 
#                       f'""{vs2022_bat}" && cl /nologo {filename_base}.cpp /Fe{filename_base}.exe"']
    
#     return ["Windows", "Unavailable", []]